### How to write cdk code in python from typescript code
this notebook is based on a [youtube video](https://www.youtube.com/watch?v=pUAgg0TzSHw)
Amazon API reference document for cdk is [here](https://docs.aws.amazon.com/cdk/api/v1/docs/aws-construct-library.html)
* most aws cdk example code is in TypeScript. How can we write python code from these examples
* first, find the function to deploy a resources used by TypeScript
* then find the corresponding function in python
* look at python documentation for that method and arguments
* the scope of python is usually self (corresponding to "this" in TypeScript)
* give an id for the resource
* The arguments in TypeScript have the corresponding componentes in Python. Just find the details of each components
  + from python document
  + make sure it is consistent to the definition in TypeScript example
  + pay attention to a lot of constants defined by cdk resources, such as dynamodb.Attribute.String etc.
    + you can get all these constants from cdk python document by clicking the "attributes" link  
* some arguments are optional, but is not documented in Python. Reference the TypeScript examples to confirm*

### Example code
```python
from aws_cdk import (
    aws_dynamodb as _dynamodb,
    aws_kinesis as _kinesis,
    core)
```
* dynamodb
```python
my_table = _dynamodb.Table(self, id = 'my_table', table_name = 'my_table'
             , partition_key = _dynamodb.Attribute(name='lastname', type=_dynamodb.AttributeType.STRING))
```
* Aws Lambda-backed APIs with lambda function
```python
# you must have a folder under cdk root directory called 'lambdacode' that contains
# a file called hello.py, which has the function called handler
my_lambda = _lambda.Function(self, id = 'lambdafunction', runtime=_lambda.Runtime.PYTHON_3_7,
                             handler = 'hello.handler',
                             code =_lambda.Code.asset('lambdacode')
my_api = _apigateway.LambdaRestApi(self, id='lambdaapi', rest_api_name='cdkapi', handler=my_lambda)
```                             

In [ ]:
## code example from https://github.com/saha-rajdeep/Python-CDK/blob/master/hello_stack.py
from aws_cdk import (
    aws_dynamodb as _dynamodb,
    aws_kinesis as _kinesis,
    aws_s3 as _s3,
    aws_lambda as _lambda,
    aws_apigateway as _apigateway,
    core
)


class MyStack(core.Stack):

    def __init__(self, scope: core.Construct, id: str, **kwargs) -> None:
        super().__init__(scope, id, **kwargs)
        
        my_table= _dynamodb.Table(self,id='dynamoTable',table_name='testcdktable'
                                  ,partition_key=_dynamodb.Attribute(name='lastname',type=_dynamodb.AttributeType.STRING))    
                                  
        my_stream= _kinesis.Stream(self,id='kinesistream',stream_name='cdkkinesisstream')
        
        my_bucket= _s3.Bucket(self,id='s3bucket',bucket_name='rajcdkbucket')
        
        my_lambda= _lambda.Function(self,id='lambdafunction',runtime=_lambda.Runtime.PYTHON_3_7,
                    handler='hello.handler', 
                    code= _lambda.Code.asset('lambdacode')) 
                    
        my_api = _apigateway.LambdaRestApi(self,id='lambdaapi',rest_api_name='cdkapi',handler=my_lambda)
        
        api_with_method = _apigateway.RestApi(self,id='restapi',rest_api_name='cdkrestapi_music')
        #music = api_with_method.root.addResource('music')
        #music.addMethod('GET')
        music = api_with_method.root.add_resource('music')
        music.add_method('GET') 
        music.add_method("DELETE", _apigateway.HttpIntegration("http://aws.amazon.com"))

#### For details fo specific AWS technique, please refer to the specific notebook in this repo